In [7]:
import torch
import numpy as np
from linear_atomic import *
import sympy as sp

In [32]:
class Permuter(torch.nn.Module):
    def __init__(self):
        super(Permuter, self).__init__()
        self.permlayer = LearnablePermutation(4)

    def forward(self, x):
        perm_matrix = self.permlayer()
        y = torch.matmul(perm_matrix,x)
        return y[0]*y[1] + y[2]*y[3]


In [33]:
permutenet = Permuter()
loss = torch.nn.L1Loss()
optimizer = torch.optim.Adam(permutenet.parameters(), lr=0.1)

In [68]:
x = np.array([1.,-2.,3.,4.])
min = np.prod(np.abs(x))
bestperm = x
for perm in sp.utilities.iterables.multiset_permutations(x):
    val = perm[0]*perm[1]+perm[2]*perm[3]
    if val < min:
        min = val
        bestperm = perm

print(bestperm, min)

x = torch.tensor(x).to(torch.float32)
y = torch.tensor([min]).to(torch.float32)

permutenet = Permuter()
loss = torch.nn.L1Loss()
optimizer = torch.optim.Adam(permutenet.parameters(), lr=0.01)

for i in range(1000):
    pred = permutenet(x)
    l = loss(pred, y)
    if i % 50 == 0:
        print(l)
    l.backward()
    optimizer.step()
    

[-2.0, 4.0, 1.0, 3.0] -5.0
tensor(20.9698, grad_fn=<MeanBackward0>)
tensor(9.5237, grad_fn=<MeanBackward0>)
tensor(3.0511, grad_fn=<MeanBackward0>)
tensor(3.1561, grad_fn=<MeanBackward0>)
tensor(4.2613, grad_fn=<MeanBackward0>)
tensor(0.0939, grad_fn=<MeanBackward0>)
tensor(1.2548, grad_fn=<MeanBackward0>)
tensor(1.0790, grad_fn=<MeanBackward0>)


/home/cculver/anaconda3/envs/generalml/lib/python3.10/site-packages/torch/nn/modules/loss.py:101: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)


tensor(1.9304, grad_fn=<MeanBackward0>)
tensor(1.1702, grad_fn=<MeanBackward0>)
tensor(1.8933, grad_fn=<MeanBackward0>)
tensor(0.4866, grad_fn=<MeanBackward0>)
tensor(0.0745, grad_fn=<MeanBackward0>)
tensor(0.5997, grad_fn=<MeanBackward0>)
tensor(0.1366, grad_fn=<MeanBackward0>)
tensor(0.0631, grad_fn=<MeanBackward0>)
tensor(0.1002, grad_fn=<MeanBackward0>)
tensor(0.0050, grad_fn=<MeanBackward0>)
tensor(0.0101, grad_fn=<MeanBackward0>)
tensor(0.0058, grad_fn=<MeanBackward0>)


In [71]:
permutenet.eval()
permutenet.train()
permutenet(x)
print(permutenet.permlayer.forward())
print()
print(torch.matmul(permutenet.permlayer.forward(),x))

tensor([[1.8072e-05, 7.4230e-12, 9.9998e-01, 4.1758e-09],
        [4.4695e-11, 1.0000e+00, 5.9108e-12, 4.2788e-11],
        [9.9935e-01, 6.5385e-04, 6.2373e-12, 2.2591e-11],
        [9.9980e-01, 1.9804e-04, 8.5603e-11, 5.7663e-10]],
       grad_fn=<SoftmaxBackward0>)

tensor([ 3.0000, -2.0000,  0.9996,  0.9998], grad_fn=<MvBackward0>)


In [66]:
permutenet.eval()
permutenet(x)

/home/cculver/code/determinism/fpna-robustness/codes/non_atomic_linear_layer/linear_atomic.py:127: UserWarning: Warning: In eval mode, not all rows and columns have exactly one '1'
  warnings.warn("Warning: In eval mode, not all rows and columns have exactly one '1'")


tensor(72.)